<a href="https://colab.research.google.com/github/scienclick/ScikitLearnPipeline4NLP/blob/master/summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## for data
import json
import pandas as pd
import numpy as np## for plotting
import matplotlib.pyplot as plt
import seaborn as sns## for processing
import re
import nltk## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing## for explainer
# from lime import lime_text## for word embedding
import gensim
import gensim.downloader as gensim_api## for deep learning
# from tensorflow.keras import models, layers, preprocessing as kprocessing
# from tensorflow.keras import backend as K## for bert language model
import sklearn.feature_selection
# import transformers
from zeugma import EmbeddingTransformer
from symspellpy import SymSpell,Verbosity
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = nltk.corpus.stopwords.words("english")
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(20,12)})
from sklearn.metrics import plot_confusion_matrix

In [ ]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())  # normalize case and remove punctuation
    tokens = word_tokenize(text)  # tokenize text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # lemmatize andremove stop words
    return tokens

## populating dictionary

In [ ]:
#populating domain dictionary
from symspellpy import SymSpell,Verbosity
path = 'new 8.txt'
symspell = SymSpell()
symspell.create_dictionary(path,encoding='utf-8')
suggestions = symspell.lookup("casing", Verbosity.CLOSEST,max_edit_distance=1, include_unknown=True)
for suggestion in suggestions:
    print(suggestion)

# str(suggestions[0]).split(",")[0]
# suggestions[0].term
# suggestions[0].distance
# suggestions[0].count


## Reading data and investigating

In [ ]:
# <editor-fold desc="Read data full ">
data = pd.read_csv("train.csv")

In [ ]:
data.head()
data.columns
data.isnull().sum()
data["Activity"].value_counts()

In [ ]:
data.groupby('Activity')["Description"].count().plot.barh(ylim=0)

In [ ]:
# percentage
target = data["Activity"]
counter = Counter(target)
for k, v in counter.items():
    per = v / len(target) * 100
    print('Class=%s\t, Count=  %d, \tPercentage=  %.3f%%                 ' % (k, v, per))

In [ ]:
data["Activity"].value_counts()
data["digits"] = data["Description"].apply(lambda y: sum(c.isdigit() for c in y))
data["letters"] = data["Description"].apply(lambda y: sum(c.isalpha() for c in y))
data["others"] = data["Description"].apply(
    lambda y: len(y) - sum(c.isalpha() for c in y) - sum(c.isdigit() for c in y) - sum(c.isspace() for c in y))

In [ ]:
ax = sns.boxplot(x="Activity", y="others",data=data)

In [ ]:
ax = sns.boxplot(x="Activity", y="digits", data=data)
# ax = sns.boxplot(x="Activity", y="letters", data=data)
# ax = sns.boxplot(x="Activity", y="Hours", data=data)
# data["Hours"].plot(kind="hist", rotation="vertical")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['Description'], data['label'], random_state=444)
print('rows in the original data set: {}'.format(data.shape[0]))
print('rows in the training set: {}'.format(X_train.shape[0]))
print('rows in the test set: {}'.format(X_test.shape[0]))

## Reading Odd words

In [ ]:
master_str=' '.join(data["Description"])
master_str=re.sub(r"[^a-zA-Z]", " ", master_str.lower())
master_tokenized=tokenize(master_str)
master_tokenized=[t for t in master_tokenized if t not in stop_words]
master= Counter(master_tokenized)
# len(master)
# master= Counter(master_tokenized).most_common()
# len(master)

odd_list=[(k,val) for k,val in master.items() if k not in symspell.words.keys()]

len(odd_list)
odd_list2=[(r[0],r[1]) for r in odd_list if symspell.lookup(r[0], Verbosity.CLOSEST,max_edit_distance=1, include_unknown=False)==[]]
len(odd_list2)

odd_list_sorted=sorted(odd_list2, key=lambda tup: tup[1])
odd_list_rare=[(r[0],r[1]) for r in odd_list2 if r[1]<2]
len(odd_list_rare)
final_odds=[r[0] for r in odd_list_rare]

odd_str=" ".join(final_odds)


len(master)
len(final_odds)



### simple models

In [ ]:
#this function is suitable when we use all the data
def evaluate_model(X, y, model):
    # define evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    # evaluate model
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores

In [ ]:
text_clf_lsvc2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC())
])

#this is good to use all the data
scores=evaluate_model(X_train,y_train,text_clf_lsvc2)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

#this is good when we put aside 20% for test
text_clf_lsvc2.fit(X_train, y_train)
predictions = text_clf_lsvc2.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
category_id_df = data[['Activity', "label"]].drop_duplicates().sort_values('label').set_index("label")
plot_confusion_matrix(text_clf_lsvc2, X_test, y_test, display_labels=list(category_id_df.Activity.values), cmap="hot",
                      xticks_rotation="vertical")

In [ ]:
plot_confusion_matrix(text_clf_lsvc2, X_test, y_test, normalize="true",
                      display_labels=list(category_id_df.Activity.values), cmap="hot", xticks_rotation="vertical",
                      values_format=".1f")


In [ ]:
plot_confusion_matrix(text_clf_lsvc2, X_test, y_test, normalize="pred",
                      display_labels=list(category_id_df.Activity.values), cmap="hot", xticks_rotation="vertical",
                      values_format=".1f")

### possible tunings

('clf', LinearSVC(class_weight="balanced"))

('vect', CountVectorizer(ngram_range=(1,3)))

('tfidf', TfidfTransformer(use_idf=True)),


## Grid search

In [ ]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC()),
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2),(1,3),(1,4)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-1,1e-2, 1e-3,1e-4),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
scores=evaluate_model(X_train,y_train,gs_clf)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

gs_clf = gs_clf.fit(X_train, y_train)
predictions = gs_clf.best_estimator_.predict(X_test)
print(classification_report(y_test, predictions))

## Feature union

In [ ]:
class Digit_Extractor(BaseEstimator, TransformerMixin):
    def text_length(self, text):
        return len(text) - text.count(" ")
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(lambda y: sum(c.isdigit() for c in y))
        return pd.DataFrame(X_tagged)


class Length_Extractor(BaseEstimator, TransformerMixin):
    def text_length(self, text):
        return len(text) - text.count(" ")
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.text_length)

        return pd.DataFrame(X_tagged)


class Hour_Extractor(BaseEstimator, TransformerMixin):
    def text_length(self, text):
        return len(text) - text.count(" ")
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        X_tagged = data.iloc[X.index]["Hours"]
        return pd.DataFrame(X_tagged)


def model_pipeline2():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer())
            ])),

            # ('digit_length', Digit_Extractor()),
            # ('entire_length', Length_Extractor()),
            ('hours', Hour_Extractor())

        ])),

        ('clf', LinearSVC())
    ])

    return pipeline


model = model_pipeline2()
model.fit(X_train, y_train);
y_pred = model.predict(X_test)
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
class num_Extractor(BaseEstimator, TransformerMixin):
    def text_length(self, text):
        text = re.sub(r"[^0-9]", " ", text.lower())  # normalize case and remove punctuation
        tokens = word_tokenize(text)  # tokenize text
        return len(tokens)/len(text)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.text_length)

        return pd.DataFrame(X_tagged)

class slash_Extractor(BaseEstimator, TransformerMixin):
    def text_length(self, text):
        text = re.sub(r"[^/]", " ", text.lower())  # normalize case and remove punctuation
        tokens = word_tokenize(text)  # tokenize text
        return len(tokens)/len(text)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.text_length)

        return pd.DataFrame(X_tagged)

class hr_Extractor(BaseEstimator, TransformerMixin):
    def text_length(self, text):
        text = re.sub(r"[^:]", " ", text.lower())  # normalize case and remove punctuation
        tokens = word_tokenize(text)  # tokenize text
        return len(tokens)/len(text)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.text_length)

        return pd.DataFrame(X_tagged)

class inch_Extractor(BaseEstimator, TransformerMixin):
    def text_length(self, text):
        text = re.sub(r"[^'\"]", " ", text.lower())  # normalize case and remove punctuation
        tokens = word_tokenize(text)  # tokenize text
        return 100*len(tokens)/len(text)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.text_length)

        return pd.DataFrame(X_tagged)

class sent_Extractor(BaseEstimator, TransformerMixin):
    def text_length(self, text):
        text = re.sub(r"[^.?!]", " ", text.lower())  # normalize case and remove punctuation
        tokens = word_tokenize(text)  # tokenize text
        return len(tokens)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.text_length)

        return pd.DataFrame(X_tagged)
    
    
def model_pipeline2():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            # ('inch', inch_Extractor()),
            ('num', num_Extractor()),
            ('slash', slash_Extractor()),
            # ('hr', hr_Extractor()),
            # ('sent', sent_Extractor()),

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=2)),
                ('tfidf', TfidfTransformer()),
            ])),

            # ('digit_length', Digit_Extractor()),
            # ('entire_length', Length_Extractor()),
            # ('hours', Hour_Extractor())



        ])),

        ('clf', LinearSVC())
    ])

    return pipeline



scores = evaluate_model(X_train, y_train, model_pipeline2())
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))


## comparison of classifiers along with grid search

In [ ]:

from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier, LogisticRegression, SGDClassifier, RidgeClassifierCV
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier, \
    VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC, NuSVC, SVC
from xgboost import XGBClassifier
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.dummy import DummyClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
classifiers = [
    LinearSVC(),
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    MultinomialNB(alpha=.001)
]
for clf in classifiers:
    try:
        print("------------------------------",text_clf_lsvc2)

        text_clf_lsvc2 = Pipeline([
            ('vect', CountVectorizer),
            ('tfidf', TfidfTransformer()),
            ('clf', clf)
        ])
        parameters = {
            'vect__ngram_range': [(1, 1), (1, 2),(1,3),(2,2),(2,3)],
            'vect__stop_words': ("english",None),
            'vect__tokenizer': (tokenize,None),
            'tfidf__smooth_idf': (True, False),
            'tfidf__use_idf': (True, False),
            'tfidf__norm': ('l1','l2'),
            # 'clf__alpha': (1e-1,1e-2, 1e-3,1e-4),
        }
        gs_clf = GridSearchCV(text_clf_lsvc2, parameters, cv=5, n_jobs=-1)
        gs_clf.fit(X_train, y_train)
        predictions = gs_clf.predict(X_test)
        print(classification_report(y_test, predictions))
        print("---------------------------------------------------------------------------")
    except:
        pass

In [ ]:
models_ = [DummyClassifier(), ExtraTreeClassifier(), MultinomialNB(), SVC(),
           BaggingClassifier(), BernoulliNB(), GaussianNB(), PassiveAggressiveClassifier(),
           LogisticRegression(solver='lbfgs', multi_class='multinomial'), SGDClassifier(), AdaBoostClassifier(),
           ExtraTreesClassifier(), RandomForestClassifier(), BaggingClassifier(), CalibratedClassifierCV(),
           LGBMClassifier(), LinearSVC(), NuSVC(), XGBClassifier(), NearestCentroid(), KNeighborsClassifier(),
           LinearDiscriminantAnalysis(), DecisionTreeClassifier(), LabelSpreading()]

for mod in models_:
    try:
        text_clf_lsvc2 = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 3), min_df=2)),
            ('tfidf', TfidfTransformer()),
            ('clf', mod)
        ])
        print("------------", mod)
        text_clf_lsvc2.fit(X_train, y_train)
        scores=evaluate_model(X_train,y_train,text_clf_lsvc2)
        print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
        # predictions = text_clf_lsvc2.predict(X_test)
        # print(classification_report(y_test, predictions))
    except Exception as e:
        print(e.args)
        pass

## stacking, ensembling

In [ ]:
model1 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=2)),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC(C=1))
])

model2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=1, max_df=0.95
                             )),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB(fit_prior=True, alpha=0.001)),
])
# model3 = Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize_no_odd, ngram_range=(1, 3), min_df=1, max_df=0.95
#                              )),
#     ('tfidf', TfidfTransformer()),
#     ('to_dense', DenseTransformer()),
#     ('clf', GaussianNB()),
# ])
model3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=1, max_df=0.95
                             )),
    ('tfidf', TfidfTransformer()),
    ('clf', ExtraTreesClassifier()),
])
model4 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=1, max_df=0.95)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])
model5 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=1, max_df=0.95)),
    ('tfidf', TfidfTransformer()),
    ('clf', CalibratedClassifierCV()),
])

estimators = []

estimators.append(('m1', model1))
estimators.append(('m2', model2))
estimators.append(('m3', model3))
estimators.append(('m4', model4))
estimators.append(('m5', model5))

level1 = LogisticRegression(solver="liblinear")

stacking = StackingClassifier(estimators=estimators, final_estimator=level1)
scores = evaluate_model(X_train, y_train, stacking)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

stacking.fit(X_train, y_train)
predictions = stacking.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
model1 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 3), min_df=2, max_df=.5, max_features=70000)),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC(C=1))
])

model2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 3), max_df=1)),
    ('tfidf', TfidfTransformer(use_idf=False, smooth_idf=True)),
    ('clf', MultinomialNB(fit_prior=True, alpha=0.001)),
])
model3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 3), max_df=1)),
    ('tfidf', TfidfTransformer(use_idf=False, smooth_idf=True)),
    ('clf', GaussianNB()),
])
model4 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 3))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])
model5 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 3))),
    ('tfidf', TfidfTransformer()),
    ('clf', CalibratedClassifierCV()),
])

estimators = []

estimators.append(('m1', model1))
estimators.append(('m2', model2))
# estimators.append(('m3', model3))
estimators.append(('m4', model4))
estimators.append(('m5', model5))

level1 = LogisticRegression(solver="liblinear")

stacking = VotingClassifier(estimators=estimators, weights=(3, 1, 1, 2))
scores = evaluate_model(X_train, y_train, stacking)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

stacking.fit(X_train, y_train)
predictions = stacking.predict(X_test)
print(classification_report(y_test, predictions))

#Grid search
stacking = VotingClassifier(estimators=estimators, weights=(3, 1, 1, 2))
ensemble = VotingClassifier(estimators)
params = {'voting': ['hard', ],
          'weights': [(2, 1, 1, 1), (1, 1, 1, 2), (2, 1, 1, 2), (3, 1, 1, 1), (3, 1, 1, 3),
                      (3, 1, 1, 2), (2, 1, 1, 3), (4, 1, 1, 2), (4, 1, 1, 3), (4, 1, 1, 4)]}

gs_clf = GridSearchCV(ensemble, params, n_jobs=4, verbose=2, cv=3)
gs_clf.fit(X_train, y_train)

results = pd.DataFrame(gs_clf.cv_results_)[["mean_test_score", "params", "rank_test_score"]]

scores = evaluate_model(X_train, y_train, ensemble)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

ensemble.fit(X_train, y_train)
predictions = ensemble.predict(X_test)
print(classification_report(y_test, predictions))

## sampler

In [ ]:
from imblearn.over_sampling import SMOTE, SMOTENC, RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as impipeline
model5 = impipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_no_odd, ngram_range=(1, 3), min_df=1)),
    ('tfidf', TfidfTransformer()),
    ('over', SVMSMOTE()),
    # ('under', RandomUnderSampler()),

    ('clf', LinearSVC()),
])
scores=evaluate_model(X_train,y_train,model5)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

model5.fit(X_train, y_train)
predictions = ensemble.predict(X_test)
print(classification_report(y_test, predictions))

## Randomized search

In [ ]:
params = {
        'vect__max_features': [10000, 25000, 40000, 70000, 100000],
        'vect__max_df': [0.5, .75, .85, 1],
        'clf__criterion': ['gini', 'entropy'],
          'clf__max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 30, 40, 50, 70, 90, 120, 150],
          "clf__min_samples_split": range(1, 10),
          "clf__min_samples_leaf": range(1, 5)}

text_clf_lsvc2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_no_odd, ngram_range=(1, 3), min_df=1)),
    ('tfidf', TfidfTransformer()),
    ('clf', DecisionTreeClassifier(ccp_alpha=.001)),
])
gs_clf = RandomizedSearchCV(text_clf_lsvc2, param_distributions=params, random_state=42, n_iter=5, cv=3, verbose=2,
                            n_jobs=4, return_train_score=True)
gs_clf.get_params().keys()
scores = evaluate_model(X_train, y_train, gs_clf)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

gs_clf.fit(X_train, y_train)
results = pd.DataFrame(gs_clf.cv_results_)[["params", "mean_test_score", "rank_test_score"]]

predictions = text_clf_lsvc2.predict(X_test)
print(classification_report(y_test, predictions))
gs_clf.best_params_

## simple Optuna

In [ ]:
import optuna
from optuna.samplers import TPESampler
def objective_func(trial):
    n = trial.suggest_int("n_estimators", 100, 500)

    model5 = impipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_no_odd, ngram_range=(1, 3), min_df=1)),
        ('tfidf', TfidfTransformer()),

        ('clf', ExtraTreesClassifier(n_estimators=n)),

    ])

    model5.fit(X_train, y_train)
    return model5.score(X_test,y_test)

study = optuna.create_study(direction='maximize',
                            sampler=TPESampler())
study.optimize(objective_func, n_trials=5)
best_trial = study.best_trial.value




def objective_func(trial):
    c = trial.suggest_loguniform("C", 1e-2, 1e+11)
    mi = trial.suggest_loguniform("max_iter", 1000, 3000)
    p = trial.suggest_categorical("penalty", "l1", "l2")
    m = trial.suggest_categorical("multi_class", "ovr", "crammer_singer")
    i = trial.suggest_loguniform("intercept_scaling", 1e-1, 2)
    w = trial.suggest_categorical("class_weight", "balanced", None)

    model5 = impipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_no_odd, ngram_range=(1, 3), min_df=1)),
        ('tfidf', TfidfTransformer()),

        ('clf', LinearSVC(C=c,max_iter=mi,penalty=p,multi_class=m,intercept_scaling=i,class_weight=w)),

    ])

    model5.fit(X_train, y_train)
    return model5.score(X_test,y_test)

study = optuna.create_study(direction='maximize',
                            sampler=TPESampler())
study.optimize(objective_func, n_trials=25)
best_trial = study.best_trial.value


## nn on best results

In [ ]:
#first extracting the results
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_predict
print(f"*************************** simple SVC")
model_svc = impipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=2)),
    ('tfidf', TfidfTransformer()),
    # ('over', BorderlineSMOTE()),
    # # ('under', RandomUnderSampler(sampling_strategy=w_under)),
    #
    # # ('select',SelectPercentile(chi2,percentile=10)),
    # ('clf', LinearSVC()),
    ('clf', LinearSVC()),
#     ('clf', OneVsRestClassifier(LinearSVC())),
    # ('clf', DecisionTreeClassifier(ccp_alpha=.001)),
    # ('clf', xgb.sklearn.XGBClassifier(nthread=-1)),
    # ('clf', xgb.XGBClassifier(treemethod="gpu_exact",random_state=42, seed=2, colsample_bytree=0.6, subsample=0.7,
    #                           n_estimators=300)),
])
scores = evaluate_model(X_train, y_train, model_svc)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))


y_pred_linear_svc = cross_val_predict(model_svc, X_train, y_train, cv=5,verbose=10,n_jobs=4)

print(f"*************************** simple MultinomialNB")
model_nb = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=1, max_df=0.95)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB( alpha=0.005)),
])
scores = evaluate_model(X_train, y_train, model_nb)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
y_pred_multinomialnb = cross_val_predict(model_nb, X_train, y_train, cv=5,verbose=10,n_jobs=4)
print(f"*************************** simple XG")
from xgboost.sklearn import XGBClassifier

model_xg = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=1, max_df=0.95)),
    ('tfidf', TfidfTransformer()),
    ('clf', XGBClassifier(
                          learning_rate =0.1,
                          n_estimators=1000,
                          max_depth=5,
                          min_child_weight=1,
                          gamma=0,
                          subsample=0.8,
                          colsample_bytree=0.8,
                          objective= 'multi:softmax',
                          nthread=2,
                          scale_pos_weight=1,
                          seed=27)),
])
scores = evaluate_model(X_train, y_train, model_xg)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

y_pred_xg = cross_val_predict(model_xg, X_train, y_train, cv=5,verbose=10,n_jobs=4)
print(f"*************************** simple SGD")
model_sgd = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=1, max_df=0.95)),
    ('tfidf', TfidfTransformer()),

    ('clf', SGDClassifier(alpha=0.000035)),
])
scores = evaluate_model(X_train, y_train, model_sgd)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
y_pred_sgd = cross_val_predict(model_sgd, X_train, y_train, cv=5,verbose=10,n_jobs=4)


print(f"*************************** simple CalibratedClassifierCV")
model5 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_snow, ngram_range=(1, 3), min_df=1, max_df=0.95)),
    ('tfidf', TfidfTransformer()),
    ('clf', CalibratedClassifierCV(base_estimator=xgb.XGBClassifier)),
])
scores = evaluate_model(X_train, y_train, model5)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
y_pred_calibrated = cross_val_predict(model5, X_train, y_train, cv=5,verbose=10,n_jobs=4)



print(f"*************************** simple ExtraTreesClassifier")
model_xtree =Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_no_odd, ngram_range=(1, 3), min_df=1, max_df=0.95
                             )),
    ('tfidf', TfidfTransformer()),
    ('clf', ExtraTreesClassifier(n_estimators= 100)),
])
scores = evaluate_model(X_train, y_train, model_xtree)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
y_pred_extratree = cross_val_predict(model_xtree, X_train, y_train, cv=5,verbose=10,n_jobs=4)

#### create nn with most repeated words

In [ ]:
res=pd.DataFrame(data={"svc":y_pred_linear_svc,"nb":y_pred_multinomialnb,"xg":y_pred_xg,"sgd":y_pred_sgd,
                       "xtree":y_pred_extratree,"y":y_train,
                       "hours":data.Hours,"len_":data["Description"].apply(lambda y:len(y)),
                       "c":data["Description"].apply(lambda y:[i[0] for i in Counter(tokenize_snow_no_digits_0(y)).most_common()][:2])
                       })
res["numc"]=data["Description"].apply(lambda y: 1 if len(tokenize_inverse(y))>0  else 0)
res["sent"]=data["Description"].apply(lambda y: 1 if  len(tokenize_sentense(y))>0 else 0)
res["hr"]=data["Description"].apply(lambda y: 1 if len(tokenize_hours(y))>0 else 0)
res["slash"]=data["Description"].apply(lambda y:1 if  len(tokenize_slash(y))>0 else 0)
res["inch"]=data["Description"].apply(lambda y: 1 if len(tokenize_inch(y))>0 else 0)

res.sample(50,random_state=7)

res["predicted"]=(res.y==res.svc) | (res.y==res.nb)| (res.y==res.xg) | (res.y==res.sgd) | (res.y==res.xtree)
res["predicted"].value_counts()
wrongs=res[res["predicted"]==False]
wrongs.sample(50)
wrongs["svc"]==6
wrongs["nb"]==6
wrongs["xg"]==6
wrongs["sgd"]==6
wrongs["xtree"]==6
wrong_6=wrongs[(wrongs["svc"]==6)&(wrongs["xg"]==6)&(wrongs["nb"]==6)&(wrongs["sgd"]==6)&(wrongs["xtree"]==6)]
wrong_6.sample(60)

cat_cols = ['svc', 'nb', 'xg','sgd','xtree']
cont_cols = ['hours','len_',"numc","sent","hr","slash","inch"]
cats = np.stack([res[col].values for col in cat_cols], 1)
import torch
cats = torch.tensor(cats, dtype=torch.int64)
res["cc"]=res["c"].apply(lambda y: " ".join(y))
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        # ('tfidf', TfidfTransformer()),
    ])
    return pipeline
model_t = model_pipeline()
transformed=model_t.fit(res.cc);
vals=transformed.transform(res.cc).toarray()
# vals
features=model_t.steps[0][1].get_feature_names()
frequency_matrix_count_0 = pd.DataFrame(vals, columns=features)
frequency_matrix_count_0
# frequency_matrix_count_0[cont_cols[0]]=res[cont_cols[0]]
# frequency_matrix_count_0[cont_cols[1]]=res[cont_cols[1]]
# frequency_matrix_count_0[cont_cols[2]]=res[cont_cols[2]]
# frequency_matrix_count_0[cont_cols[3]]=res[cont_cols[3]]
# frequency_matrix_count_0[cont_cols[4]]=res[cont_cols[4]]
# frequency_matrix_count_0[cont_cols[5]]=res[cont_cols[5]]
# frequency_matrix_count_0[cont_cols[6]]=res[cont_cols[6]]
conts=frequency_matrix_count_0
conts = torch.tensor(conts.values, dtype=torch.float)
cats.shape
conts.shape
emb_szs = [(24,32),(24,32),(24,32),(24,32),(24,32)]
emb_szs
import torch.nn as nn
class TabularModel(nn.Module):
    def __init__(self, emb_szs, n_cont, out_sz, layers, p=0.5):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(p)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        layerlist = []
        n_emb = sum((nf for ni,nf in emb_szs))
        n_in = n_emb + n_cont
        for i in layers:
            layerlist.append(nn.Linear(n_in,i))
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
        self.layers = nn.Sequential(*layerlist)
    def forward(self, x_cat, x_cont):
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.emb_drop(x)
        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x, x_cont], 1)
        x = self.layers(x)
        return x
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(33)
model = TabularModel(emb_szs, conts.shape[1], 24, [400,200,100,50], p=0.5) # out_sz = 2
criterion = nn.CrossEntropyLoss()
criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model.to(device)
batch_size = 20419
test_size = 4000
cat_train = cats[:batch_size-test_size]
cat_test = cats[batch_size-test_size:batch_size]
con_train = conts[:batch_size-test_size]
con_test = conts[batch_size-test_size:batch_size]
yy_train = data["label"][:-test_size] if test_size!=0 else data["label"]
yy_test = data["label"][-test_size:]
yy_train = torch.tensor(yy_train.values, dtype=torch.int64,device=device)
len(cat_train)
len(yy_train)
len(con_train)
cat_train.shape
import time
start_time = time.time()
epochs = 1800
# epochs = 10000
losses = []
optimizer.zero_grad()
for i in range(epochs):
    i+=1
    y_pred = model(cat_train.to(device), con_train.to(device))
    y_pred.size
    loss = criterion(y_pred, yy_train)
    losses.append(loss)
    # a neat trick to save screen space:
    if i%100 == 1:
        print(f'epoch: {i:3}  loss: {loss.item():10.8f}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(f'epoch: {i:3}  loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed
with torch.no_grad():
    preds=model.cpu()(cat_test,con_test)
    preds.shape
    t,i=torch.max(nn.functional.log_softmax(preds,dim=1),1)
    # t,i=torch.max(preds,1)
    print(classification_report(yy_test, i.cpu()))

##  storing results

In [ ]:
data_test = pd.read_excel("/test.xlsx")
X_ = data_test["Description"]
pd.DataFrame({"prediction": stacking.predict(X_)}).to_csv("zz.csv")

In [ ]:
data_test = pd.read_excel("test.xlsx")
X_ = data_test["Description"]
model_svc.fit(X_train,y_train)
model_xg.fit(X_train,y_train)
model_nb.fit(X_train,y_train)
model_sgd.fit(X_train,y_train)
model_xtree.fit(X_train,y_train)

svc_=model_svc.predict(X_)
xg_=model_xg.predict(X_)
nb_=model_nb.predict(X_)
sgd_=model_sgd.predict(X_)
xtree_=model_xtree.predict(X_)

res_test=pd.DataFrame(data={"svc":svc_,"nb":nb_,"xg":xg_,"sgd":sgd_,"xtree":xtree_,
                       "c":data_test["Description"].apply(lambda y:[i[0] for i in Counter(tokenize_snow_no_digits_0(y)).most_common()][:2])
                       })

res_test.to_csv("/level2_test.csv")




cat_cols = ['svc', 'nb', 'xg','sgd','xtree']
cats_pred = np.stack([res_test[col].values for col in cat_cols], 1)
import torch
cats_pred = torch.tensor(cats_pred, dtype=torch.int64)

res_test["cc"]=res_test["c"].apply(lambda y: " ".join(y))
vals_test=transformed.transform(res_test.cc).toarray()
# vals
features=model_t.steps[0][1].get_feature_names()
frequency_matrix_count_0 = pd.DataFrame(vals_test, columns=features)
frequency_matrix_count_0
# frequency_matrix_count_0[cont_cols[0]]=res[cont_cols[0]]
# frequency_matrix_count_0[cont_cols[1]]=res[cont_cols[1]]
conts_pred=frequency_matrix_count_0
conts_pred = torch.tensor(conts_pred.values, dtype=torch.float)
cats_pred.shape
conts_pred.shape


y_pred=model.cpu()(cats_pred,conts_pred)
with torch.no_grad():
    t,i=torch.max(nn.functional.log_softmax(y_pred.cpu(),dim = 1),1)
    pd.DataFrame({"prediction": i}).to_csv("/torch.csv")